### Load and read about the data:
* https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [8]:
df = pd.read_csv("D:\Makeen\ProjectML\train.csv")

OSError: [Errno 22] Invalid argument: 'D:\\Makeen\\ProjectML\train.csv'

In [5]:
df.count()

PhraseId      156060
SentenceId    156060
Phrase        156060
Sentiment     156060
dtype: int64

In [6]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [7]:
df['Phrase'].head(10)

0    A series of escapades demonstrating the adage ...
1    A series of escapades demonstrating the adage ...
2                                             A series
3                                                    A
4                                               series
5    of escapades demonstrating the adage that what...
6                                                   of
7    escapades demonstrating the adage that what is...
8                                            escapades
9    demonstrating the adage that what is good for ...
Name: Phrase, dtype: object

In [8]:
df['Sentiment'].describe()

count    156060.000000
mean          2.063578
std           0.893832
min           0.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           4.000000
Name: Sentiment, dtype: float64

In [8]:
print(df['Sentiment'].value_counts())

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64


In [10]:
df['Sentiment'].value_counts()/df['Sentiment'].count()

Sentiment
2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: count, dtype: float64

In [14]:
X, y = df['Phrase'], df['Sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5)

In [16]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression())
])

In [17]:
parameters = {
    'vect__max_df': (0.25, 0.5),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}

In [18]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='accuracy')

In [19]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/root/miniconda/envs/py310-env1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/root/miniconda/envs/py310-env1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': (0.1, 1, 10), 'vect__max_df': (0.25, 0.5),
                         'vect__ngram_range': ((1, 1), (1, 2)),
                         'vect__use_idf': (True, False)},
             scoring='accuracy', verbose=1)

In [20]:
print('Best score: %0.3f' % grid_search.best_score_)

Best score: 0.624


In [21]:
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('t%s: %r' % (param_name, best_parameters[param_name]))

Best parameters set:
tclf__C: 10
tvect__max_df: 0.25
tvect__ngram_range: (1, 2)
tvect__use_idf: False


In [22]:
predictions = grid_search.predict(X_test)

In [23]:
print('Accuracy: %s' % accuracy_score(y_test, predictions))

Accuracy: 0.6252851467384339


In [24]:
print('Confusion Matrix:')
print(confusion_matrix(y_test, predictions))

Confusion Matrix:
[[ 1070  1575   862   100    12]
 [  865  5403  6747   632    36]
 [  211  2886 33011  3396   208]
 [   25   447  6966  7758  1207]
 [    3    37   619  2405  1549]]


In [25]:
print('Classification Report:')
print(classification_report(y_test, predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.30      0.37      3619
           1       0.52      0.39      0.45     13683
           2       0.68      0.83      0.75     39712
           3       0.54      0.47      0.51     16403
           4       0.51      0.34      0.41      4613

    accuracy                           0.63     78030
   macro avg       0.55      0.47      0.50     78030
weighted avg       0.61      0.63      0.61     78030



# Save grid search object
* experiment to load it on nlp_read_grid_search.ipynb notebook

In [27]:
type(grid_search)

sklearn.model_selection._search.GridSearchCV

In [28]:
grid_search.cv_results_

{'mean_fit_time': array([ 49.95999618,  88.33991079, 214.12229066, 223.53946109,
         73.16855583,  69.61061926, 130.25862508,  97.92665277,
         31.2739356 ,  33.62529516,  87.14433513,  87.46246219,
         35.60416431,  22.58561735,  66.66689014,  65.79807734,
         25.29822679,  26.84719515,  66.64608641,  66.05116963,
         27.2559442 ,  27.18639154, 133.04586515, 113.96779585]),
 'std_fit_time': array([ 2.41921451,  8.29453777, 19.39666828, 32.80617061,  6.27691445,
         5.39613922,  8.05151996, 21.17030565,  4.38926106,  3.43516721,
         4.04597022, 11.63443   ,  3.07922451,  1.37984594,  4.19131415,
         1.77535006,  0.43448068,  0.28250459,  1.43759366,  1.62910945,
         1.24964655,  2.27941434, 14.22550404, 38.39658963]),
 'mean_score_time': array([4.86111016, 4.21728673, 8.48813982, 7.88218703, 5.46173267,
        4.51872783, 2.86931067, 2.77931561, 1.99421692, 1.1636241 ,
        5.51447148, 3.86374412, 1.14907551, 1.6260313 , 2.23090515,
    

In [30]:
import pickle
pickle.dump(grid_search, open("./nlp_basics_2_grid_search.pkl", "wb"))